In [ ]:
from Classification.modelmodule import ModelModuleCLS
from Classification.datamodule import DataModuleCLS

%run Classification/mainfittest.py --config StressLevelPrediction/configs/stress.yaml

# %run Classification/mainfittest.py --config StressLevelPrediction/configs/stress.yaml --config StressLevelPrediction/configs/stress.local.yaml

In [1]:
# Direct Inference Example (Without API)
# This demonstrates how to use the trained model for predictions

import joblib
import onnxruntime as ort
import numpy as np
import pandas as pd
from pathlib import Path

# Setup paths
MODELS_DIR = Path("StressLevelPrediction/models")

print("=" * 80)
print("Loading Models for Inference")
print("=" * 80)

# Load preprocessor
print("\n1. Loading preprocessor...")
preprocessor = joblib.load(MODELS_DIR / "preprocessor.joblib")
print("   ✓ Preprocessor loaded")

# Load label encoder (if exists)
print("\n2. Loading label encoder...")
label_encoder_path = MODELS_DIR / "label_encoder.joblib"
if label_encoder_path.exists():
    label_encoder = joblib.load(label_encoder_path)
    print("   ✓ Label encoder loaded")
    print(f"   Class names: {label_encoder.classes_.tolist()}")
else:
    label_encoder = None
    print("   ⚠ Label encoder not found (will use 0-indexed mapping)")

# Load ONNX model
print("\n3. Loading ONNX model...")
onnx_model_path = MODELS_DIR / "stress_level_model.onnx"
session = ort.InferenceSession(str(onnx_model_path))
print("   ✓ ONNX model loaded")

print("\n" + "=" * 80)
print("Running Inference Examples")
print("=" * 80)

# Define feature columns (must match training config)
categorical_cols = ['Work_From', 'Work_Life_Balance', 'Lives_With_Family', 'Working_State']
numeric_cols = [
    'Avg_Working_Hours_Per_Day', 'Work_Pressure', 'Manager_Support',
    'Sleeping_Habit', 'Exercise_Habit', 'Job_Satisfaction', 'Social_Person'
]
feature_cols = categorical_cols + numeric_cols

# Example 1: Sample employee data
print("\n" + "-" * 80)
print("Example 1: Sample Employee Prediction")
print("-" * 80)

sample_data = pd.DataFrame({
    'Work_From': ['Home'],
    'Work_Life_Balance': ['Yes'],
    'Lives_With_Family': ['Yes'],
    'Working_State': ['Karnataka'],
    'Avg_Working_Hours_Per_Day': [8.5],
    'Work_Pressure': [3],
    'Manager_Support': [4],
    'Sleeping_Habit': [4],
    'Exercise_Habit': [3],
    'Job_Satisfaction': [4],
    'Social_Person': [3],
})

print("\nInput Features:")
print(sample_data[feature_cols].to_string(index=False))

# Transform data
transformed = preprocessor.transform(sample_data[feature_cols])
print(f"\nTransformed shape: {transformed.shape}")

# Predict
input_name = session.get_inputs()[0].name
output = session.run(None, {input_name: transformed.astype(np.float32)})
logits = output[0][0]

# Apply softmax to get probabilities
exp_logits = np.exp(logits - np.max(logits))  # Numerical stability
probabilities = exp_logits / np.sum(exp_logits)
predicted_class_idx = int(np.argmax(logits))

# Map back to original label
if label_encoder:
    predicted_label = int(label_encoder.inverse_transform([predicted_class_idx])[0])
else:
    predicted_label = predicted_class_idx + 1

# Stress level labels
stress_labels = {
    1: "Low",
    2: "Low-Medium",
    3: "Medium",
    4: "Medium-High",
    5: "High"
}

print("\n" + "=" * 80)
print("PREDICTION RESULT")
print("=" * 80)
print(f"\nPredicted Stress Level: {predicted_label} ({stress_labels.get(predicted_label, 'Unknown')})")
print(f"Confidence: {probabilities[predicted_class_idx] * 100:.2f}%")

print("\nAll Class Probabilities:")
print("-" * 80)
for i, prob in enumerate(probabilities):
    level = i + 1 if label_encoder is None else int(label_encoder.inverse_transform([i])[0])
    label = stress_labels.get(level, "Unknown")
    print(f"Level {level} ({label:15s}): {prob * 100:6.2f}% {'█' * int(prob * 50)}")

# Example 2: Multiple predictions from dataset
print("\n\n" + "=" * 80)
print("Example 2: Predicting on Real Dataset Samples")
print("=" * 80)

# Load actual dataset
dataset_path = Path("StressLevelPrediction/data/stress_level.csv")
if dataset_path.exists():
    df = pd.read_csv(dataset_path)
    
    # Take first 3 samples
    test_samples = df.head(3)
    
    print(f"\nTesting on {len(test_samples)} samples from dataset:")
    print("-" * 80)
    
    for idx, row in test_samples.iterrows():
        # Prepare input
        input_data = pd.DataFrame([row[feature_cols]])
        
        # Transform and predict
        transformed = preprocessor.transform(input_data[feature_cols])
        output = session.run(None, {input_name: transformed.astype(np.float32)})
        logits = output[0][0]
        
        # Get prediction
        exp_logits = np.exp(logits - np.max(logits))
        probabilities = exp_logits / np.sum(exp_logits)
        predicted_class_idx = int(np.argmax(logits))
        
        if label_encoder:
            predicted_label = int(label_encoder.inverse_transform([predicted_class_idx])[0])
        else:
            predicted_label = predicted_class_idx + 1
        
        actual_label = int(row['Stress_Level'])
        
        print(f"\nSample {idx + 1}:")
        print(f"  Work From: {row['Work_From']}, Hours: {row['Avg_Working_Hours_Per_Day']:.1f}h")
        print(f"  Work Pressure: {row['Work_Pressure']}/5, Manager Support: {row['Manager_Support']}/5")
        print(f"  Actual Stress Level: {actual_label} ({stress_labels.get(actual_label, 'Unknown')})")
        print(f"  Predicted Stress Level: {predicted_label} ({stress_labels.get(predicted_label, 'Unknown')})")
        print(f"  Confidence: {probabilities[predicted_class_idx] * 100:.2f}%")
        print(f"  {'✓ Correct' if actual_label == predicted_label else '✗ Incorrect'}")

print("\n" + "=" * 80)
print("Inference Complete!")
print("=" * 80)


Loading Models for Inference

1. Loading preprocessor...
   ✓ Preprocessor loaded

2. Loading label encoder...
   ✓ Label encoder loaded
   Class names: [1, 2, 3, 4, 5]

3. Loading ONNX model...
   ✓ ONNX model loaded

Running Inference Examples

--------------------------------------------------------------------------------
Example 1: Sample Employee Prediction
--------------------------------------------------------------------------------

Input Features:
Work_From Work_Life_Balance Lives_With_Family Working_State  Avg_Working_Hours_Per_Day  Work_Pressure  Manager_Support  Sleeping_Habit  Exercise_Habit  Job_Satisfaction  Social_Person
     Home               Yes               Yes     Karnataka                        8.5              3                4               4               3                 4              3

Transformed shape: (1, 15)

PREDICTION RESULT

Predicted Stress Level: 1 (Low)
Confidence: 22.64%

All Class Probabilities:
-------------------------------------------